In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#!conda install -c conda-forge geopy --yes 
# uncomment this line if you haven't completed the Foursquare API lab
!pip install geocoder
from geopy.geocoders import Nominatim
#!conda install -c conda-forge folium=0.5.0 --yes 
# uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium 

     |████████████████████████████████| 102kB 6.6MB/s ta 0:00:011
     |████████████████████████████████| 102kB 8.0MB/s ta 0:00:011


In [2]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(data, 'lxml')

In [3]:
#scraping and creating a dataframe

table = soup.find_all('table')
df = pd.read_html(str(table))[0]
df.head()
#df.shape()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
#dropping rows with "Not assigned" Boroughs

df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
# Check if a cell has a borough but a Not assigned neighborhood

if sum(df["Neighborhood"].astype("str").str.contains("Not assigned")) > 0:
    print("found")
else:
    print("not found")

not found


In [6]:
df3 = pd.read_csv("http://cocl.us/Geospatial_data")
df3.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
# df2 is the final dataframe consisiting of required columns

df2= pd.merge(df,df3,on='Postal Code')
df2.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [84]:
df.shape

(103, 3)

In [8]:
class color:
    BOLD = '\033[1m'
    END = '\033[0m'
print(color.BOLD + 'EXPLORING & CLUSTERING IN TORONTO' + color.END)

EXPLORING & CLUSTERING IN TORONTO


In [9]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
print((location.latitude, location.longitude))

(43.6534817, -79.3839347)


In [10]:
latitude = location.latitude
longitude = location.longitude
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [11]:
df_MainBorough = df2['Borough'].value_counts(ascending=False)
df_MainBorough

# We can see that we have 4 Boroughs with Toronto name in it. we'll analyse them.

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
York                 5
East York            5
Mississauga          1
Name: Borough, dtype: int64

In [12]:
# create a df for Borough names with Toronto

Toronto_Borough = df2[df2['Borough'].str.contains("Toronto")]
Toronto_Borough.head()
Toronto_Borough['Borough'].value_counts()

Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

In [14]:
address = 'Central Toronto, TO'
latitude_DT = location.latitude
longitude_DT = location.longitude
geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
print((location.latitude, location.longitude))

(43.6534817, -79.3839347)


In [15]:
# create map of all Toronto boroughs using latitude and longitude values
map_Torontos = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_Borough['Latitude'], Toronto_Borough['Longitude'], Toronto_Borough['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Torontos)  
    
map_Torontos

In [18]:
#@hidden_cell

from pandas.io.json import json_normalize

radius = 1000
LIMIT = 30

CLIENT_ID = 'IVOC3IMFG1KW21LUODLTVB5SZYZGMUGWPDGZSH1A2DIYP2WC' #  Foursquare ID
CLIENT_SECRET = 'U41NKKZMMPEBTHL4XX2PMNSECHBQ2VQOPKSM5TUBSIA321HK' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: IVOC3IMFG1KW21LUODLTVB5SZYZGMUGWPDGZSH1A2DIYP2WC
CLIENT_SECRET:U41NKKZMMPEBTHL4XX2PMNSECHBQ2VQOPKSM5TUBSIA321HK


In [19]:
url = 'https://api.foursquare.com/v2/venues/search?client_id=IVOC3IMFG1KW21LUODLTVB5SZYZGMUGWPDGZSH1A2DIYP2WC&client_secret=U41NKKZMMPEBTHL4XX2PMNSECHBQ2VQOPKSM5TUBSIA321HK&ll=43.6534817,-79.3839347&v=20180604&radius=1000&limit=30'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=IVOC3IMFG1KW21LUODLTVB5SZYZGMUGWPDGZSH1A2DIYP2WC&client_secret=U41NKKZMMPEBTHL4XX2PMNSECHBQ2VQOPKSM5TUBSIA321HK&ll=43.6534817,-79.3839347&v=20180604&radius=1000&limit=30'

In [20]:
Toronto_Borough.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed4d3c89c6f59001b157e40'},
 'response': {'venues': [{'id': '4c093ee0340720a153728493',
    'name': 'City Hall Council Chambers',
    'location': {'lat': 43.65182710471462,
     'lng': -79.38394893163043,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65182710471462,
       'lng': -79.38394893163043}],
     'distance': 184,
     'cc': 'CA',
     'country': 'Canada',
     'formattedAddress': ['Canada']},
    'categories': [{'id': '4bf58dd8d48988d129941735',
      'name': 'City Hall',
      'pluralName': 'City Halls',
      'shortName': 'City Hall',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/cityhall_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1591006332',
    'hasPerk': False},
   {'id': '4ad4c05ef964a5208ff620e3',
    'name': 'Toronto City Hall',
    'location': {'address': '100 Queen St. W.',
     'crossStreet': 'at Bay St.',
     'lat': 43.65313989695342,
     'lng': -79.

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['venues']

nearby_venues = json_normalize(venues)

nearby_venues.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",False,4c093ee0340720a153728493,NaN,CA,NaN,Canada,NaN,184,[Canada],"[{'label': 'display', 'lat': 43.65182710471462...",43.651827,-79.383949,NaN,NaN,City Hall Council Chambers,v-1591006332
1,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",False,4ad4c05ef964a5208ff620e3,100 Queen St. W.,CA,Toronto,Canada,at Bay St.,38,"[100 Queen St. W. (at Bay St.), Toronto ON M5H...","[{'label': 'display', 'lat': 43.65313989695342...",43.653140,-79.383967,M5H 2N2,ON,Toronto City Hall,v-1591006332
2,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",False,5b193c42598e64002ca79b96,100 Queen St W,CA,Toronto,Canada,NaN,3,"[100 Queen St W, Toronto ON M5H 2N2, Canada]","[{'label': 'display', 'lat': 43.653454, 'lng':...",43.653454,-79.383952,M5H 2N2,ON,City of Toronto Civic Innovation Office,v-1591006332
3,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,50885719498ea7b5aab3a74c,483 Bay St,CA,Toronto,Canada,NaN,130,"[483 Bay St, Toronto ON M5G 2C9, Canada]","[{'label': 'display', 'lat': 43.653436, 'lng':...",43.653436,-79.382314,M5G 2C9,ON,GoodLife Fitness Toronto Bell Trinity Centre,v-1591006332
4,"[{'id': '4bf58dd8d48988d15a941735', 'name': 'G...",False,4c0121fd9a950f47fa9208c6,100 Queen Street West,CA,Toronto,Canada,NaN,6,"[100 Queen Street West, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65350358617817...",43.653504,-79.383866,NaN,ON,City Hall Podium Green Roof,v-1591006332


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

30 venues were returned by Foursquare.


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:

Toronto_Borough_venues = getNearbyVenues(names=Toronto_Borough['Neighborhood'],
                                   latitudes=Toronto_Borough['Latitude'],
                                   longitudes=Toronto_Borough['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [27]:
Toronto_Borough_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [28]:
Toronto_Borough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",18,18,18,18,18,18
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,30,30,30,30,30,30
Christie,16,16,16,16,16,16
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


In [29]:
print('There are {} uniques categories.'.format(len(Toronto_Borough_venues['Venue Category'].unique())))

There are 194 uniques categories.


In [55]:
for col in Toronto_Borough_venues.columns: 
    print(col) 

Neighborhood
Neighborhood Latitude
Neighborhood Longitude
Venue
Venue Latitude
Venue Longitude
Venue Category


In [62]:
#Toronto_Borough_venues1 = Toronto_Borough_venues
#Toronto_onehot = pd.get_dummies(Toronto_Borough_venues1["Venue Category"], dummy_na=True)
#Toronto_onehot = pd.concat([Toronto_Borough_venues1, Toronto_onehot], axis=1)

#Toronto_onehot = Toronto_onehot.drop(['Neighborhood Latitude'], axis = 1) 

#Toronto_onehot.head()

toronto_denc_onehot = pd.get_dummies(Toronto_Borough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_denc_onehot['Neighborhood'] = Toronto_Borough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_denc_onehot.columns[-1]] + list(toronto_denc_onehot.columns[:-1])
toronto_denc_onehot = toronto_denc_onehot[fixed_columns]

toronto_denc_onehot.head()


,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
toronto_denc_onehot.shape

(863, 194)

In [60]:
for col in toronto_denc_onehot.columns: 
    print(col) 

Yoga Studio
Airport
Airport Food Court
Airport Gate
Airport Lounge
Airport Service
Airport Terminal
American Restaurant
Aquarium
Art Gallery
Art Museum
Arts & Crafts Store
Asian Restaurant
Auto Workshop
BBQ Joint
Baby Store
Bagel Shop
Bakery
Bank
Bar
Basketball Stadium
Beer Bar
Beer Store
Belgian Restaurant
Bistro
Board Shop
Boat or Ferry
Bookstore
Boutique
Breakfast Spot
Brewery
Bubble Tea Shop
Burger Joint
Burrito Place
Bus Line
Butcher
Café
Cajun / Creole Restaurant
Candy Store
Caribbean Restaurant
Cheese Shop
Chinese Restaurant
Chocolate Shop
Climbing Gym
Clothing Store
Cocktail Bar
Coffee Shop
College Arts Building
College Gym
College Rec Center
Colombian Restaurant
Comfort Food Restaurant
Comic Shop
Concert Hall
Convenience Store
Cosmetics Shop
Coworking Space
Creperie
Cuban Restaurant
Dance Studio
Deli / Bodega
Department Store
Dessert Shop
Diner
Discount Store
Distribution Center
Dog Run
Donut Shop
Eastern European Restaurant
Electronics Store
Ethiopian Restaurant
Event Space
F

In [63]:

Toronto_grouped = toronto_denc_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.066667,0.066667,0.066667,0.066667,0.133333,0.066667,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,Central Bay Street,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0


In [67]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Seafood Restaurant,Beer Bar,Coffee Shop,Cocktail Bar,Tailor Shop,Museum,Basketball Stadium,Farmers Market,Café,Bakery
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Stadium,Italian Restaurant,Restaurant,Climbing Gym,Nightclub,Bar,Convenience Store
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Brewery,Farmers Market,Recording Studio,Butcher,Restaurant,Auto Workshop,Spa,Burrito Place,Fast Food Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Boat or Ferry,Coffee Shop,Boutique,Rental Car Location,Bar,Plane,Harbor / Marina,Sculpture Garden,Airport Gate
4,Central Bay Street,Coffee Shop,Café,Park,Seafood Restaurant,Sandwich Place,Bubble Tea Shop,Ramen Restaurant,Poke Place,Chinese Restaurant,Yoga Studio


In [68]:
# We'll cluster Toronto borough venues 

k=5
toronto_clustering = Toronto_grouped.drop(['Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_


array([1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 3, 1, 2, 1,
       1, 1, 1, 0, 3, 4, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1], dtype=int32)

In [76]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_denc_merged = Toronto_Borough

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_denc_merged = toronto_denc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_denc_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Breakfast Spot,Theater,Bakery,Distribution Center,Dessert Shop,Restaurant,Event Space,Spa
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Wings Joint,Distribution Center,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Hobby Shop,Gym
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Café,Coffee Shop,Theater,Shopping Mall,Burger Joint,Hotel,Bar,Electronics Store,College Rec Center,Sandwich Place
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Gastropub,Café,Cocktail Bar,Coffee Shop,Vegetarian / Vegan Restaurant,BBQ Joint,Jazz Club,Japanese Restaurant,Italian Restaurant,New American Restaurant
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Coffee Shop,Trail,Pub,Health Food Store,Dance Studio,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store


In [77]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        toronto_denc_merged['Latitude'], 
        toronto_denc_merged['Longitude'], 
        toronto_denc_merged['Neighborhood'], 
        toronto_denc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [78]:
#Examine clsuters 0 to 4: Total k=5

toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 0, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Downtown Toronto,0,Café,Coffee Shop,Theater,Shopping Mall,Burger Joint,Hotel,Bar,Electronics Store,College Rec Center,Sandwich Place
15,Downtown Toronto,0,Gastropub,Café,Cocktail Bar,Coffee Shop,Vegetarian / Vegan Restaurant,BBQ Joint,Jazz Club,Japanese Restaurant,Italian Restaurant,New American Restaurant
24,Downtown Toronto,0,Coffee Shop,Café,Park,Seafood Restaurant,Sandwich Place,Bubble Tea Shop,Ramen Restaurant,Poke Place,Chinese Restaurant,Yoga Studio
30,Downtown Toronto,0,Coffee Shop,Café,Colombian Restaurant,Pizza Place,Japanese Restaurant,Seafood Restaurant,Deli / Bodega,Smoke Shop,Hotel,Speakeasy
42,Downtown Toronto,0,Coffee Shop,Café,Deli / Bodega,Japanese Restaurant,Gym / Fitness Center,Sandwich Place,Restaurant,Bookstore,Museum,Beer Bar
43,West Toronto,0,Café,Coffee Shop,Breakfast Spot,Stadium,Italian Restaurant,Restaurant,Climbing Gym,Nightclub,Bar,Convenience Store
48,Downtown Toronto,0,Café,Gastropub,Coffee Shop,Deli / Bodega,Restaurant,Japanese Restaurant,Steakhouse,Pub,Sandwich Place,Seafood Restaurant
54,East Toronto,0,Café,Coffee Shop,Bakery,Yoga Studio,Park,Bookstore,Seafood Restaurant,Sandwich Place,Brewery,Cheese Shop
74,Central Toronto,0,Café,Sandwich Place,Coffee Shop,Indian Restaurant,BBQ Joint,Shoe Repair,Pub,Liquor Store,Burger Joint,History Museum
92,Downtown Toronto,0,Restaurant,Seafood Restaurant,Café,Cocktail Bar,Beer Bar,Comfort Food Restaurant,Tailor Shop,Jazz Club,Japanese Restaurant,Farmers Market


In [79]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 1, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1,Coffee Shop,Park,Breakfast Spot,Theater,Bakery,Distribution Center,Dessert Shop,Restaurant,Event Space,Spa
4,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Wings Joint,Distribution Center,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Hobby Shop,Gym
19,East Toronto,1,Coffee Shop,Trail,Pub,Health Food Store,Dance Studio,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store
20,Downtown Toronto,1,Seafood Restaurant,Beer Bar,Coffee Shop,Cocktail Bar,Tailor Shop,Museum,Basketball Stadium,Farmers Market,Café,Bakery
25,Downtown Toronto,1,Grocery Store,Café,Park,Restaurant,Diner,Italian Restaurant,Baby Store,Candy Store,Coffee Shop,Nightclub
31,West Toronto,1,Pharmacy,Bakery,Supermarket,Music Venue,Middle Eastern Restaurant,Pool,Café,Brewery,Bar,Bank
36,Downtown Toronto,1,Hotel,Plaza,Park,Performing Arts Venue,Café,Deli / Bodega,Lake,Sporting Goods Shop,Basketball Stadium,IT Services
37,West Toronto,1,Bar,Asian Restaurant,Men's Store,Yoga Studio,Brewery,Café,Record Shop,Pizza Place,Coffee Shop,New American Restaurant
41,East Toronto,1,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Bubble Tea Shop,Bookstore,Juice Bar,Restaurant,Spa,Dessert Shop
47,East Toronto,1,Fast Food Restaurant,Park,Sandwich Place,Pet Store,Movie Theater,Pizza Place,Pub,Liquor Store,Restaurant,Burrito Place


In [80]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 2, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,2,Gym,Trail,Tennis Court,Wings Joint,Dance Studio,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store


In [81]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 3, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,3,Bus Line,Park,Swim School,Wings Joint,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
91,Downtown Toronto,3,Park,Playground,Trail,Wings Joint,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner


In [82]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 4, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,4,Garden,Music Venue,Home Service,Wings Joint,Dance Studio,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store
